<a href="https://colab.research.google.com/github/MohsenMaaleki/Electricity-Consumption-in-Malmi-Office-Building/blob/main/Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.subplots as sp
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor



In [158]:
df = pd.read_csv("/content/drive/MyDrive/Energy_Weather_data/Malmi_office_building_hourly.csv")

In [181]:
print("5 first rows of dataset")
df.head()

5 first rows of dataset


,Date,ElCons,Heat,Cloud amount (1/8),Pressure (msl) (hPa),Relative humidity (%),Air temperature (degC),Dew-point temperature (degC),Horizontal visibility (m),Wind direction (deg),Gust speed (m/s),Wind speed (m/s)
0,2020-01-01 00:00:00,17.25,70,0,1011.5,90,-1.9,-3.3,18632,226,2.6,2.0
1,2020-01-01 01:00:00,17.21,70,0,1011.4,92,-2.1,-3.1,22284,237,2.9,2.1
2,2020-01-01 02:00:00,17.00,80,0,1010.9,87,-0.5,-2.5,50000,286,4.0,2.6
3,2020-01-01 03:00:00,17.22,70,0,1010.7,83,0.2,-2.4,48969,274,4.2,2.9
4,2020-01-01 04:00:00,17.09,70,3,1009.9,84,0.1,-2.2,49724,244,4.2,3.0


#Data Preprocessing

### Hourly electricity, heating consumption and weather-related variables taken from the office building.

In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1848 entries, 0 to 1847
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Date                          1848 non-null   object 
 1   ElCons                        1848 non-null   float64
 2   Heat                          1848 non-null   int64  
 3   Cloud amount (1/8)            1848 non-null   int64  
 4   Pressure (msl) (hPa)          1848 non-null   float64
 5   Relative humidity (%)         1848 non-null   int64  
 6   Air temperature (degC)        1848 non-null   float64
 7   Dew-point temperature (degC)  1848 non-null   float64
 8   Horizontal visibility (m)     1848 non-null   int64  
 9   Wind direction (deg)          1848 non-null   int64  
 10  Gust speed (m/s)              1848 non-null   float64
 11  Wind speed (m/s)              1848 non-null   float64
dtypes: float64(6), int64(5), object(1)
memory usage: 173.4+ KB


In [182]:
print("Describe numberical columns")
df.describe()

Describe numberical columns


,ElCons,Heat,Cloud amount (1/8),Pressure (msl) (hPa),Relative humidity (%),Air temperature (degC),Dew-point temperature (degC),Horizontal visibility (m),Wind direction (deg),Gust speed (m/s),Wind speed (m/s)
count,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000
mean,29.476228,68.517316,5.769481,1002.096266,84.895022,1.677489,-0.743669,29447.894481,204.626082,7.145671,4.446266
std,16.449097,15.735659,3.211360,11.808863,12.560490,3.289017,4.190069,17203.921827,79.803218,3.779383,2.326791
min,11.740000,40.000000,0.000000,964.000000,28.000000,-12.900000,-16.300000,222.000000,0.000000,0.000000,0.000000
25%,15.880000,60.000000,4.000000,996.400000,80.000000,0.400000,-2.800000,13324.500000,174.000000,4.600000,2.900000
50%,19.105000,70.000000,8.000000,1002.600000,88.000000,2.400000,0.500000,29558.500000,214.000000,6.900000,4.300000
75%,41.262500,80.000000,8.000000,1010.800000,94.000000,3.900000,2.300000,49407.750000,256.000000,9.400000,5.800000
max,66.490000,140.000000,9.000000,1026.300000,100.000000,7.900000,6.500000,50000.000000,360.000000,20.300000,11.700000


In [180]:
df.isnull().sum()

Date                            0
ElCons                          0
Heat                            0
Cloud amount (1/8)              0
Pressure (msl) (hPa)            0
Relative humidity (%)           0
Air temperature (degC)          0
Dew-point temperature (degC)    0
Horizontal visibility (m)       0
Wind direction (deg)            0
Gust speed (m/s)                0
Wind speed (m/s)                0
dtype: int64

In [163]:
#Converting Date column to datatime format
df['Date'] = pd.to_datetime(df['Date'])


In [164]:
#Scaling data using StandardScaler

# Select the columns to be scaled
columns_to_scale = ['ElCons', 'Heat', 'Cloud amount (1/8)', 'Pressure (msl) (hPa)',
                    'Relative humidity (%)', 'Air temperature (degC)',
                    'Dew-point temperature (degC)', 'Horizontal visibility (m)',
                    'Wind direction (deg)', 'Gust speed (m/s)', 'Wind speed (m/s)']

# Create a StandardScaler object
scaler = StandardScaler()

# Fit and transform the selected columns
df_scaled = scaler.fit_transform(df[columns_to_scale])

# Create a new DataFrame with scaled columns
df_scaled = pd.DataFrame(df_scaled, columns=columns_to_scale)

# Concatenate the scaled columns with the remaining columns from df
df_scaled = pd.concat([df[['Date']], df_scaled], axis=1)


In [165]:
df_scaled.head()

,Date,ElCons,Heat,Cloud amount (1/8),Pressure (msl) (hPa),Relative humidity (%),Air temperature (degC),Dew-point temperature (degC),Horizontal visibility (m),Wind direction (deg),Gust speed (m/s),Wind speed (m/s)
0,2020-01-01 00:00:00,-0.743478,0.094250,-1.797071,0.796544,0.406541,-1.088002,-0.610258,-0.628858,0.267905,-1.203080,-1.051632
1,2020-01-01 01:00:00,-0.745910,0.094250,-1.797071,0.788074,0.565814,-1.148827,-0.562513,-0.416523,0.405782,-1.123681,-1.008643
2,2020-01-01 02:00:00,-0.758680,0.729921,-1.797071,0.745721,0.167633,-0.662228,-0.419279,1.194941,1.019958,-0.832549,-0.793697
3,2020-01-01 03:00:00,-0.745302,0.094250,-1.797071,0.728780,-0.150912,-0.449341,-0.395406,1.134996,0.869548,-0.779616,-0.664729
4,2020-01-01 04:00:00,-0.753207,0.094250,-0.862635,0.661016,-0.071276,-0.479753,-0.347661,1.178894,0.493521,-0.779616,-0.621740


#Exploratory Data Analysis (EDA)

In [166]:
#Separate Date to year, Month, day and hour
df_scaled['Date'] = pd.to_datetime(df_scaled['Date'])

# Extract Year, Month, and Day
df_scaled['Year'] = df_scaled['Date'].dt.year
df_scaled['Month'] = df_scaled['Date'].dt.month
df_scaled['Day'] = df_scaled['Date'].dt.day
df_scaled['Hour'] = df_scaled['Date'].dt.hour

# Drop the original 'Date' column
df_scaled = df_scaled.drop('Date', axis=1)
df_scaled.head()

,ElCons,Heat,Cloud amount (1/8),Pressure (msl) (hPa),Relative humidity (%),Air temperature (degC),Dew-point temperature (degC),Horizontal visibility (m),Wind direction (deg),Gust speed (m/s),Wind speed (m/s),Year,Month,Day,Hour
0,-0.743478,0.094250,-1.797071,0.796544,0.406541,-1.088002,-0.610258,-0.628858,0.267905,-1.203080,-1.051632,2020,1,1,0
1,-0.745910,0.094250,-1.797071,0.788074,0.565814,-1.148827,-0.562513,-0.416523,0.405782,-1.123681,-1.008643,2020,1,1,1
2,-0.758680,0.729921,-1.797071,0.745721,0.167633,-0.662228,-0.419279,1.194941,1.019958,-0.832549,-0.793697,2020,1,1,2
3,-0.745302,0.094250,-1.797071,0.728780,-0.150912,-0.449341,-0.395406,1.134996,0.869548,-0.779616,-0.664729,2020,1,1,3
4,-0.753207,0.094250,-0.862635,0.661016,-0.071276,-0.479753,-0.347661,1.178894,0.493521,-0.779616,-0.621740,2020,1,1,4


In [167]:
# Histogram: Yearly Heating Consumption

fig = px.histogram(df_scaled, x='ElCons', color='Year', nbins=30,
                   title='Yearly Heating Consumption')
fig.update_layout(xaxis_title='Heating Consumption', yaxis_title='Count')
fig.show()


In [168]:
# Line Plot: Hourly Heating Consumption
hourly_consumption = df_scaled.groupby('Hour')['ElCons'].mean().reset_index()

fig = px.line(hourly_consumption, x='Hour', y='ElCons', title='Hourly Heating Consumption')
fig.update_layout(xaxis_title='Hour', yaxis_title='Average Heating Consumption')
fig.show()


In [169]:
fig = make_subplots(rows=5, cols=1, subplot_titles=("Heating Consumption", "Air Temperature", "Cloud Amount","Wind speed" ,"ElCons"))

# Add histograms to subplots
fig.add_trace(go.Histogram(x=df['Heat'], nbinsx=30), row=1, col=1)
fig.add_trace(go.Histogram(x=df['Air temperature (degC)'], nbinsx=30), row=2, col=1)
fig.add_trace(go.Histogram(x=df['Cloud amount (1/8)'], nbinsx=30), row=3, col=1)
fig.add_trace(go.Histogram(x=df['Wind speed (m/s)'], nbinsx=30), row=4, col=1)
fig.add_trace(go.Histogram(x=df['ElCons'], nbinsx=30), row=5, col=1)

# Update the layout
fig.update_layout(
    title="Histograms of Selected Variables",
    height=800,
    showlegend=False
)

# Update subplot titles and axis labels
fig.update_xaxes(title_text="Heating Consumption", row=1, col=1)
fig.update_xaxes(title_text="Air Temperature (degC)", row=2, col=1)
fig.update_xaxes(title_text="Cloud Amount (1/8)", row=3, col=1)
fig.update_xaxes(title_text='Wind speed (m/s)', row=4, col=1)
fig.update_xaxes(title_text='ElCons', row=5, col=1)

fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Count", row=2, col=1)
fig.update_yaxes(title_text="Count", row=3, col=1)

# Show the plot
fig.show()


In [170]:
# Line Plot: Monthly Heating Consumption
monthly_consumption = df_scaled.groupby('Month')['ElCons'].mean().reset_index()

fig = px.line(monthly_consumption, x='Month', y='ElCons', title='Monthly Heating Consumption')
fig.update_layout(xaxis_title='Month', yaxis_title='Average Heating Consumption')
fig.show()


In [171]:
# Select only numeric columns for correlation calculation
numeric_columns = df_scaled.select_dtypes(include=['float64', 'int64']).columns

# Calculate the correlation matrix
correlation_matrix = df_scaled[numeric_columns].corr()

# Create a correlation heatmap using Plotly
fig = px.imshow(correlation_matrix, color_continuous_scale='RdBu', title='Correlation Heatmap')

# Update layout
fig.update_layout(
    xaxis_title='Features',
    yaxis_title='Features',
    width=800,
    height=600
)

# Show the correlation heatmap
fig.show()


In [172]:
# Select the columns for visualization
columns_to_visualize = ['ElCons', 'Heat', 'Air temperature (degC)', 'Relative humidity (%)']

# Create subplots
fig = sp.make_subplots(rows=len(columns_to_visualize), cols=len(columns_to_visualize), shared_xaxes=False, shared_yaxes=False)

# Loop through each column combination
for i, col1 in enumerate(columns_to_visualize):
    for j, col2 in enumerate(columns_to_visualize):
        # Add scatter plot for each variable combination
        if i == j:
            fig.add_trace(go.Histogram(x=df[col1], name=col1), row=i+1, col=j+1)
        else:
            fig.add_trace(go.Scatter(x=df[col1], y=df[col2], mode='markers', name=f'{col1} vs {col2}'), row=i+1, col=j+1)

# Update layout
fig.update_layout(title='Pairplot-like Visualization')

# Show the figure
fig.show()


#Machine Learning

In [173]:
# Select the features and target variable
features = df_scaled.drop('ElCons', axis=1)
target = df_scaled['ElCons']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Verify the sizes of the training and testing sets
print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)


Training set size: (1478, 14)
Testing set size: (370, 14)


## Linear Regression

In [174]:
# Create an instance of LinearRegression
lr_model = LinearRegression()

# Train the model
lr_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = lr_model.predict(X_test)

# Evaluate the model
lr_mse = mean_squared_error(y_test, y_pred)
lr_mae = mean_absolute_error(y_test, y_pred)
lr_r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print("LinearRegression Mean Squared Error (MSE):", lr_mse)
print("LinearRegression Mean Absolute Error (MAE):", lr_mae)
print("LinearRegression R-squared Score:", lr_r2)

LinearRegression Mean Squared Error (MSE): 0.6347616689440313
LinearRegression Mean Absolute Error (MAE): 0.6666399662800906
LinearRegression R-squared Score: 0.21864529532677202


## Decision Tree

In [175]:
# Create an instance of DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state=42)

# Train the model
dt_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = dt_model.predict(X_test)

# Evaluate the model
dt_mse = mean_squared_error(y_test, y_pred)
dt_mae = mean_absolute_error(y_test, y_pred)
dt_r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print("DecisionTreeRegressor Mean Squared Error (MSE):", dt_mse)
print("DecisionTreeRegressor Mean Absolute Error (MAE):", dt_mae)
print("DecisionTreeRegressor R-squared Score:", dt_r2)

DecisionTreeRegressor Mean Squared Error (MSE): 0.32663039462601207
DecisionTreeRegressor Mean Absolute Error (MAE): 0.2773300537024078
DecisionTreeRegressor R-squared Score: 0.5979369769525096


## Random Forest

In [176]:
# Create the Random Forest Regression model
rf_model = RandomForestRegressor(random_state=42)

# Train the model on the training data
rf_model .fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_model.predict(X_test)

# Evaluate the model
rf_mse = mean_squared_error(y_test, y_pred)
rf_mae = mean_absolute_error(y_test, y_pred)
rf_r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print("RandomForestRegressor Mean Squared Error (MSE):", rf_mse)
print("RandomForestRegressor Mean Absolute Error (MAE):", rf_mae)
print("RandomForestRegressor R-squared Score:", rf_r2)


RandomForestRegressor Mean Squared Error (MSE): 0.15584062546891694
RandomForestRegressor Mean Absolute Error (MAE): 0.25394529668219534
RandomForestRegressor R-squared Score: 0.8081692517887478


## Gradient Boostin

In [177]:
# Train the Gradient Boosting model
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = gb_model.predict(X_test)

# Evaluate the model
gb_mse = mean_squared_error(y_test, y_pred)
gb_mae = mean_absolute_error(y_test, y_pred)
gb_r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print("Gradient Boosting Mean Squared Error (MSE):", gb_mse)
print("Gradient Boosting Mean Absolute Error (MAE):", gb_mae)
print("Gradient Boosting R-squared Score:", gb_r2)


Gradient Boosting Mean Squared Error (MSE): 0.23338129099059593
Gradient Boosting Mean Absolute Error (MAE): 0.3477038428630692
Gradient Boosting R-squared Score: 0.7127212013265212


In [178]:
# Define the evaluation metrics for each model
models = ['Linear Regression', 'Decision Tree', 'Random Forest', 'Gradient Boosting']
mse = [lr_mse, dt_mse, rf_mse, gb_mse]
mae = [lr_mae, dt_mae, rf_mae, gb_mae]
r2 = [lr_r2, dt_r2, rf_r2, gb_r2]

# Create the bar plots for MSE, MAE, and R-squared
fig = go.Figure(data=[
    go.Bar(name='MSE', x=models, y=mse),
    go.Bar(name='MAE', x=models, y=mae),
    go.Bar(name='R-squared', x=models, y=r2)
])

# Update layout
fig.update_layout(
    title='Model Performance Comparison',
    xaxis_title='Models',
    yaxis_title='Metric Value',
    barmode='group'
)

# Show the figure
fig.show()
